In [4]:
import os
from dotenv import load_dotenv
from autogen import ConversableAgent, GroupChat, GroupChatManager

# --- Configuration ---
load_dotenv()
llm_config = {"model": "gpt-4o-mini"}

# --- Initial Message ---
product_owner_message = """
Hello team, I'm the Product Owner.

Here is the customer's request:
“ I want to create a web-based mobile app for our insurance company where my customers can input their budget, the reason why they need the insurance, and their status (e.g., student, adult). After the app gets all the information, we can provide the best insurance plan for them, and also provide the information about why we recommend this insurance plan.”

Scrum Master, please lead the team to create a product backlog and estimate the effort so we can create a sprint plan.
"""

# --- Agent Definitions ---

product_owner = ConversableAgent(
    name="product_owner",
    llm_config=llm_config,
    system_message="You are the Product Owner. Your job is to provide the customer's vision. You will work with the Scrum Master to help the team break down the request into a Product Backlog (a list of features). Clarify any questions the team has about the features."
)

### UPDATED AGENT ###
# The Scrum Master now has the explicit 4-stage process in its system prompt.
scrum_master = ConversableAgent(
    name="scrum_master",
    llm_config=llm_config,
    system_message="""
You are the Scrum Master and you are the facilitator of this meeting. Your goal is to guide the team to create a full project estimate by following the 4 stages of a Sprint.

Here are the 4 STAGES you must follow:
1. Sprint Goal: Work with the Product Owner to discuss and define the Product Backlog for the project. Then, guide the team to estimate the effort for each item in the backlog.
2. Scrum Team: Oversee the Development Team's estimation process. Ensure every engineer provides an estimate for each backlog item.
3. Sprint Review: After estimations are complete, announce that it is time for the 'Planner' to create the sprint schedule.
4. Sprint Retrospective: After the Planner presents the final plan, you can briefly ask the team for feedback before the conversation ends.

Keep the team focused on this process.
"""
)

planner = ConversableAgent(
    name="Planner",
    llm_config=llm_config,
    system_message="You are the Planner. You must remain silent until the Scrum Master explicitly asks you to create the final plan. Once asked, your task is to analyze the entire conversation to find the total estimated hours for each Product Backlog item. Then, create a final sprint plan based on this data. Assume each sprint is 2 weeks long and the team's total work capacity is 400 hours per sprint (5 engineers * 40 hours/week * 2 weeks). Present the final plan clearly and then say 'TERMINATE'."
)


# --- Engineer Prompts with Embedded Knowledge Base ---

requirement_engineer = ConversableAgent(
    name="requirement_engineer",
    llm_config=llm_config,
    system_message="""
You are a Requirement Engineer on the Development Team. When the Scrum Master asks for an effort estimate for a feature, provide your estimated hours for the requirement-related tasks. Base your estimate on your professional judgment and the historical data provided below. State your estimate clearly (e.g., 'For the Login Page, I estimate 40 hours for requirements.').

**Your Historical Quality Record:**
Quality record for Requirement task
HW1
                                  Amount of work      Productivity Rate
Write requirements                156 req             5 req/hour
Review requirement
    Preparation for review        156 req             8 req/hour
    Review Meeting                156 req             8 req/hour
Rework                            97 defects          5 defects/hour
HW2
                                  Amount of work      Productivity Rate
Write requirements                165 req             4 req/hour
Review requirement
    Preparation for review        165 req             5 req/hour
    Review Meeting                165 req             10 req/hour
Rework                            210 defects         8 defects/hour
"""
)

system_engineer = ConversableAgent(
    name="system_engineer",
    llm_config=llm_config,
    system_message="""
You are a System Engineer on the Development Team. When the Scrum Master asks for an effort estimate for a feature, provide your estimated hours for the design-related tasks. Base your estimate on your professional judgment and the historical data provided below. State your estimate clearly (e.g., 'For the Login Page, I estimate 50 hours for design.').

**Your Historical Quality Record:**
Quality record for Design task
HW1
                                  Amount of work      Productivity Rate
Write Design Document (DD)        134 pages           5 pages/hour
Review DD
    Preparation for DD            134 pages           3 pages/hour
    Review DD                     134 pages           6 pages/hour
Rework                            88 defects          5 defects/hour
HW2
                                  Amount of work      Productivity Rate
Write Design Document (DD)        10 pages            6 pages/hour
Review DD
    Preparation for DD            10 pages            5 pages/hour
    Review DD                     10 pages            10 pages/hour
Rework                            278 defects         10 defects/hour
"""
)

software_engineer = ConversableAgent(
    name="software_engineer",
    llm_config=llm_config,
    system_message="""
You are a Software Engineer on the Development Team. When the Scrum Master asks for an effort estimate for a feature, provide your estimated hours for programming-related tasks. Base your estimate on your professional judgment and the historical data provided below. State your estimate clearly (e.g., 'For the Login Page, I estimate 120 hours for programming.').

**Your Historical Quality Record:**
Quality record Programming task
HW1
                                  Amount of work      Productivity Rate
Write code                        3945 SLOC           5 SLOC/hour
Unit Testing
    Prepare/Execute Test Cases    187 test case       3 test case/hour
    Fix Found Defects             155 defects         5 defects/day
    Test Fixed Defects            155 defects         10 defects/day
Code Inspection
    Preparation for review        3945 SLOC           125 SLOC/hour
    Review Meeting                3945 SLOC           200 SLOC/hour
Rework                            314 defects         3 defects/hour
HW2
                                  Amount of work      Productivity Rate
Write code                        3450 SLOC           10 SLOC/hour
Unit Testing
    Prepare/Execute Test Cases    174 test case       5 test case/hour
    Fix Found Defects             104 defects         4 defects/day
    Test Fixed Defects            104 defects         5 defects/day
Code Inspection
    Preparation for review        3450 SLOC           110 SLOC/hour
    Review Meeting                3450 SLOC           160 SLOC/hour
Rework                            309 defects         5 defects/hour
"""
)

testing_engineer = ConversableAgent(
    name="testing_engineer",
    llm_config=llm_config,
    system_message="""
You are a Testing Engineer on the Development Team. When the Scrum Master asks for an effort estimate for a feature, provide your estimated hours for testing-related tasks. Base your estimate on your professional judgment and the historical data provided below. State your estimate clearly (e.g., 'For the Login Page, I estimate 60 hours for testing.').

**Your Historical Quality Record:**
Quality record for Testing task
HW1
                                  Amount of work      Productivity Rate
Write test plan (TP)              203 pages           6 pages/day
Review TP
    Preparation for TP            203 pages           2 pages/hour
    Review TP Meeting             203 pages           6 pages/hour
    Rework                        102 defects         5 defects/day
Execute TP (test cases)           310 test case       6 test case/day
Fix Found Defects                 185 defects         12 defects/day
HW2
                                  Amount of work      Productivity Rate
Write test plan (TP)              149 pages           5 pages/day
Review TP
    Preparation for TP            149 pages           5 pages/hour
    Review TP Meeting             149 pages           6 pages/hour
    Rework                        99 defects          4 defects/day
Execute TP (test cases)           138 test case       30 test case/day
Fix Found Defects                 77 defects          4 defects/day
"""
)

documentation_engineer = ConversableAgent(
    name="documentation_engineer",
    llm_config=llm_config,
    system_message="""
You are a Documentation Engineer on the Development Team. When the Scrum Master asks for an effort estimate for a feature, provide your estimated hours for documentation-related tasks. Base your estimate on your professional judgment and the historical data provided below. State your estimate clearly (e.g., 'For the Login Page, I estimate 30 hours for documentation.').

**Your Historical Quality Record:**
Quality record Documentation task
HW1
                                  Amount of work      Productivity Rate
User Documentation                141 pages           5 pages/hour
Review UD
    Preparation for review        141 pages           4 pages/hour
    Review Meeting                141 pages           10 pages/hour
Rework                            203 defects         10 defects/hour
HW2
                                  Amount of work      Productivity Rate
User Documentation                195 pages           4 pages/hour
Review UD
    Preparation for review        195 pages           5 pages/hour
    Review Meeting                195 pages           8 pages/hour
Rework                            344 defects         15 defects/hour
"""
)


# --- Group Chat Setup ---
groupchat = GroupChat(
    agents=[
        product_owner,
        scrum_master,
        requirement_engineer,
        system_engineer,
        software_engineer,
        testing_engineer,
        documentation_engineer,
        planner
    ],
    messages=[],
    max_round=30,
    speaker_selection_method="auto"
)

manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# --- Initiate Chat ---
product_owner.initiate_chat(
    manager,
    message=product_owner_message
)

product_owner (to chat_manager):


Hello team, I'm the Product Owner.

Here is the customer's request:
“ I want to create a web-based mobile app for our insurance company where my customers can input their budget, the reason why they need the insurance, and their status (e.g., student, adult). After the app gets all the information, we can provide the best insurance plan for them, and also provide the information about why we recommend this insurance plan.”

Scrum Master, please lead the team to create a product backlog and estimate the effort so we can create a sprint plan.


--------------------------------------------------------------------------------

Next speaker: scrum_master


>>>>>>>> USING AUTO REPLY...
scrum_master (to chat_manager):

Thank you, Product Owner, for sharing the customer’s request. Let’s start defining our Product Backlog together.

### Stage 1: Sprint Goal
To create a productive Product Backlog, we need to break down the customer's request into smaller, actio

ChatResult(chat_id=None, chat_history=[{'content': "\nHello team, I'm the Product Owner.\n\nHere is the customer's request:\n“ I want to create a web-based mobile app for our insurance company where my customers can input their budget, the reason why they need the insurance, and their status (e.g., student, adult). After the app gets all the information, we can provide the best insurance plan for them, and also provide the information about why we recommend this insurance plan.”\n\nScrum Master, please lead the team to create a product backlog and estimate the effort so we can create a sprint plan.\n", 'role': 'assistant', 'name': 'product_owner'}, {'content': "Thank you, Product Owner, for sharing the customer’s request. Let’s start defining our Product Backlog together.\n\n### Stage 1: Sprint Goal\nTo create a productive Product Backlog, we need to break down the customer's request into smaller, actionable items. These will typically include user stories that capture functionality fr